In [2]:
import snap, datetime, json
from collections import Counter, defaultdict 
import random
import os

As snap dose not support python3, using python2 kernel in jupyter instead.

In [3]:
def write_json(d, fname):
    with open(fname,'w') as f:
        f.write(json.dumps(d))

def load_json(fname):
    with open(fname) as f:
        return json.load(f)


From the hotels and users file, picked out
the IDs and then assigned a unique ID for each
of the HotelID and UserID which is a running
sequence so that there is better clarity when
we look at the graph data file

In [15]:
class IdsAsInt():
    '''give node unique id in graph
    in snap, each node should have an unique id,
    edges could or could not have unique id in 
    both directed and undirected graph'''
    def __init__(self):
        self.x = -1
        self.n = {}
    def __getitem__(self,key):
        if key not in self.n:
            self.x +=1
            self.n[key] = self.x
        return self.n[key]
nid = IdsAsInt()
print nid[1]
print nid[2]
print nid[3]
print nid[3]

0
1
2
2


In [5]:
def convert_date(dic):
    date = dic['Date']
    return datetime.date(*map(int, date.split(" ")[0].split("-")))


Once IDs were
created then created the graph.txt file for
both test and train data which will contain all
the edges in the current dataset represented by
nodes on either side.

note: reviews.json have folowing variables:
##### userID：
1. Ratings
2. Author name or User name
3. Author Location
4. Date of Review
5. HotelID
6. Hotel Location

In [25]:
def create_final_json(nid, nids, inp, out):
    return write_json({nid(x):y for x,y in load_json(inp).iteritems() if nid(x) in nids},out)


def make_graph(t1,out):
    '''t1 as a time split between train and test data set'''
    nid = IdsAsInt()
    reviews_it = {}
    
    with open("./data/created/review.json") as f:
        reviews_it = json.load(f)
    nids = set() #set of unique node ids created from IdsAsInt class
    for key,review in reviews_it.iteritems():
        if convert_date(review) < t1: # time split for train and test data
            nids.add(nid['u' + review['Author']])
            nids.add(nid['h' + review['HotelID']])
    create_final_json(lambda ud: nid['u'+ud], nids, './data/created/users.json', out + 'user.json')
    create_final_json(lambda hd: nid['h'+hd], nids, './data/created/hotels.json', out + 'hotels.json')
    
    with open(out + 'graph.txt','w') as graph:
        review_data = defaultdict(lambda: defaultdict(list))
        for key, review in reviews_it.iteritems():
            ukey = nid['u'+review['Author']]
            hkey = nid['h' + review['HotelID']]
            if ukey in nids and hkey in nids:
                date = convert_date(review)
                if date <t1:
                    review_data[ukey][hkey].append(review)
                    graph.write("{:} {:}\n".format(ukey,hkey))
        for r in review_data:
            for ur in review_data[r]:
                review_data[r][ur] = sorted(review_data[r][ur],key=convert_date,reverse=True)
        write_json(review_data,out + "review.json")

In [26]:
if __name__ == '__main__':
    make_graph(datetime.date(2011,2,15),'./data/train/')
    make_graph(datetime.date(2012,2,15),'./data/test/')